In [ ]:
# install youtubedl library
# sudo -H pip install --upgrade youtube-dl

In [ ]:
import os

%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

## download youtube video

# Detecting emotions

In [2]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import time
import os

%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

ERROR:root:Line magic function `%install_ext` not found.


In [3]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = 'f96758ee1dab457b86ffc1da4205d6dc'  ######### Here you have to paste your primary key for EMOTION API
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('6a96532956704d31a587e859fad4df30') ######### set the key for FACE API 

time: 4.51 ms


In [4]:
# Image path
urlImage= ''

# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 66.5 ms


In [5]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 13.4 ms


In [23]:
# generate a fake url list to replicate S3 condition

#### ATTNTION ----- change '4641' to match the number of last image+1 -----#######
#### ATTNTION ----- change '1_23_17/1_23_17___%d.jpg' to match the path and names on your S3 bucket -----#######

url_list = []
for i in range(0,3054,1):
    url = "https://s3.amazonaws.com/100daysofspice/3_30_17/3_30_17___%d.jpg" % i
    url_list.append(url)

time: 8.14 ms


In [ ]:
# Face Detection & Emotion Analysis - UPDATED
processed = {}
face_detection = {}
count = 0

for single_image in url_list:
 
    try:
        face_detection[count] = detect(single_image)

        json = { 'url': single_image } 
        processed[count] = processRequest(json, data, headers, params)  
    except:
        pass

    try:
        if face_detection[count][0]['faceRectangle'] == processed[count][0]['faceRectangle']:
            face_detection[count][0].update(processed[count][0])
            face_detection[count][1].update(processed[count][1])
        if  face_detection[count][0]['faceRectangle'] == processed[count][1]['faceRectangle']:
            face_detection[count][0].update(processed[count][1])
            face_detection[count][1].update(processed[count][0])
    except:
        pass
    
    count += 1
    
#     time.sleep(3)

In [7]:
face_detection

{0: [], 1: [], 2: [], 3: []}

time: 14.3 ms


In [ ]:
# Save the result as Json
#### ATTNTION ----- update '1_23_17.json' to the name of your current folder #######

import json
with open('/Users/Amiros/GitHub/100_days_of_spice/json/3_30_17.json', 'w') as fp:
    json.dump(face_detection, fp, sort_keys=True)

## json to csv

In [ ]:
import json
import csv

In [ ]:
"""
This can probably be improved but serves the purpose of our analysis.
I used Chris's schema so it will be useful for Quarz analysis.

"""

EMOTIONS = [
    'contempt',
    'fear',
    'sadness',
    'disgust',
    'anger',
    'neutral',
    'happiness',
    'surprise',
]


CSV_COLUMNS = [
    'url',
    'frame_time',
    'sean_happiness',
    'sean_neutral',
    'sean_anger',
    'sean_contempt',
    'sean_suprise',
    'sean_sadness',
    'sean_fear'
]

In [ ]:
with open('/Users/Amiros/GitHub/100_days_of_spice/json/3_30_17.json') as data_file:    
    json_data = json.load(data_file)
    
json_data = {int(k):v for k,v in json_data.items()} #converting keys to integer   
frame_keys = sorted(json_data.keys())

In [ ]:
csv_data = []
   
for key in frame_keys:
    url = 'https://s3.amazonaws.com/100daysofspice/3_30_17/3_30_17___%d.jpg' % key
    frame = json_data[key]
        
    row = {
        #'frame': frame,
        'url': url,
        'frame_time': key,
    }
    
    if len(frame) > 1:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'sean_'
                try:
                    for emotion in EMOTIONS:
                        row[prefix + emotion] = face['scores'][emotion]
                except:
                    pass
                
            row.pop("frame", None)        
      
    elif len(frame) == 1:
        for face in frame:
            if face['faceAttributes']['gender'] == 'male':
                prefix = 'sean_'
                try:
                    for emotion in EMOTIONS:
                    #print(face['scores'][emotion])
                        row[prefix + emotion] = face['scores'][emotion]
                except:
                    pass
                ### assign emotions vale to each candidate and change the name to the respective one 
          
        row.pop("frame", None)        

    csv_data.append(row)

In [ ]:
import pandas as pd
df = pd.DataFrame(csv_data)

In [ ]:
df['data'] = '3_30_17'

In [ ]:
df.to_csv('csv/3_30_17-EMOTIONS.csv', sep=',', index=False)

##### 